<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/FinALBERT/fine_tuning_ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install tensorflow
!pip install transformers
!pip3 install albert-tensorflow
!pip install torch
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 890kB 17.4MB/s 
     |████████████████████████████████| 1.1MB 18.1MB/s 
     |████████████████████████████████| 3.0MB 28.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7aceb359875e5a113a3b4100f89e9d914f4e2a2ce05ed7eaaf52e10e7dfa0b06
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 81kB 2.3MB/s 


# Check GPU

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


# Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/mjag7682/ALBERT

Cloning into 'ALBERT'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 362 (delta 2), reused 0 (delta 0), pack-reused 353
Receiving objects: 100% (362/362), 244.39 KiB | 470.00 KiB/s, done.
Resolving deltas: 100% (235/235), done.


In [ ]:
!pip install -r /content/ALBERT/requirements.txt 

     |████████████████████████████████| 110.5MB 65kB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 3.8MB 41.1MB/s 
     |████████████████████████████████| 512kB 29.1MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=da09531e91a678dff87f3e427990e2f4fae4e07e33f032b7f927656d5c7d3e26
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflo

# Fine Tune

In [ ]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 320.4MB 50kB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
# !pip install modeling

ERROR: Could not find a version that satisfies the requirement modeling (from versions: none)
ERROR: No matching distribution found for modeling


In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import tensorflow as tf
# from albert import modeling
# import tokenization
# import optimization
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [ ]:
from transformers import AlbertTokenizer
from transformers.modeling_albert import AlbertModel, load_tf_weights_in_albert, AlbertPreTrainedModel
from transformers import AlbertForSequenceClassification,AlbertConfig
from transformers.tokenization_bert import BertTokenizer
import torch.nn as nn
from torch.nn import CrossEntropyLoss
VOCAB_FILE = "/content/gdrive/My Drive/ALBERTimplementation/data/30k-clean.model" # This is the vocab file output from Build Vocab step
CONFIG_FILE = "/content/gdrive/My Drive/ALBERTimplementation/AG_News/albert_config.json"
ALBERT_PRETRAIN_CHECKPOINT = "/content/gdrive/My Drive/ALBERTimplementation/AG_News/model.ckpt-best.index" # This is the model checkpoint output from Albert Pretrain step
tokenizer = AlbertTokenizer(vocab_file=VOCAB_FILE)
config = AlbertConfig.from_json_file(CONFIG_FILE)
model = AlbertModel(config)
model = load_tf_weights_in_albert(model, config,ALBERT_PRETRAIN_CHECKPOINT)

In [ ]:
# from transformers import AlbertTokenizer
# from transformers import AlbertForSequenceClassification,AlbertConfig
# config = modeling.AlbertConfig.from_json_file("/content/gdrive/My Drive/ALBERTimplementation/model-fine/config.json")
# tokenizer = tokenization.FullTokenizer.from_scratch(vocab_file="/content/gdrive/My Drive/ALBERTimplementation/model-fine/vocab.txt", do_lower_case=True, spm_model_file=None)
# tokenizer =  AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True) 
# tokenizer =  AlbertTokenizer.from_pretrained('/content/gdrive/My Drive/ALBERTimplementation/model-fine', do_lower_case=True) 
# tokenizer =  AlbertTokenizer.from_pretrained('./content/drive/My Drive/Reuters_Dataset/reut2-021', do_lower_case=True) 
# PRE_TRAINED_MODEL_NAME_OR_PATH = '/content/gdrive/My Drive/ALBERTimplementation/model-fine'
# model = AlbertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME_OR_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False)

In [ ]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf

In [ ]:
train_data = pd.read_csv('/content/gdrive/My Drive/ALBERTimplementation/sentiment_data/train.tsv', sep='\t')
for item in train_data.iterrows():
  print(item[1][1])
  if item[1][1] != 1 and item[1][1] != 0:
    print(item[1][1])

In [ ]:
train_data = pd.read_csv('/content/gdrive/My Drive/ALBERTimplementation/sentiment_data/train.tsv', sep='\t')
print("Number of training examples {}".format(len(train_data)))
num_examples = 100000
train = train_data[:num_examples].text.values
labels = train_data[:num_examples].label.values
# train = train_data[]

Number of training examples 131173


In [ ]:
print(len(train))
print(len(labels))
# print(labels[:100])
for i in labels:
  if i!=0 and i!=1:
    print(i)

100000
100000


In [ ]:
# Print the original sentence.
print(' Original: ', train[10])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train[10]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train[10])))

 Original:  vix spy new high vix new low spy calls versus lod
Tokenized:  ['▁vi', 'x', '▁spy', '▁new', '▁high', '▁vi', 'x', '▁new', '▁low', '▁spy', '▁call', 's', '▁vers', 'us', '▁lo', 'd']
Token IDs:  [1847, 782, 1181, 30, 141, 1847, 782, 30, 385, 1181, 172, 12, 3770, 595, 2947, 27]


In [ ]:
print(train_data.text.apply(lambda x: len(x)).quantile([0.9]))
MAX_LEN = 160

0.9    160.0
Name: text, dtype: float64


In [ ]:
m_l = 0
for x in train:
  if len(x)>m_l:
    m_l = len(x)
print(m_l)

1892


In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for text in train:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', train[10])
print('Token IDs:', input_ids[10])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  vix spy new high vix new low spy calls versus lod
Token IDs: tensor([   2, 1847,  782, 1181,   30,  141, 1847,  782,   30,  385, 1181,  172,
          12, 3770,  595, 2947,   27,    3,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0

In [ ]:
#training & validation split
from torch.utils.data import TensorDataset, random_split


# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{} training samples'.format(train_size))
print('{} validation samples'.format(val_size))

90000 training samples
10000 validation samples


In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Set the seed value all over the place to make this reproducible.
import random
def set_random(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

In [ ]:
def train_model(train_dataloader, optimizer, epochs):
    
    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training

        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            loss, logits = model(b_input_ids, 
                                 attention_mask=b_input_mask, 
                                 labels=b_labels)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)            

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                (loss, logits) = model(b_input_ids, 
                                       attention_mask=b_input_mask,
                                       labels=b_labels)

            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids)


        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    
    return training_stats

def print_training_stats(training_stats):
    # Display floats with two decimal places.
    pd.set_option('precision', 2)

    # Create a DataFrame from our training statistics.
    df_stats = pd.DataFrame(data=training_stats)

    # Use the 'epoch' as the row index.
    df_stats = df_stats.set_index('epoch')

    # A hack to force the column headers to wrap.
    #df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

    # Display the table.
    print(df_stats)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup, AdamW

from transformers import AlbertForSequenceClassification,AlbertConfig
# from transformers import DistilBertForSequenceClassification, AdamW, DistilBertConfig
# from transformers import BertForSequenceClassification, BertConfig
# from transformers import ElectraForSequenceClassification

# ADJUST lr_s and batch_sizes
lr_s = [2e-5]
batch_sizes = [32]
from itertools import product
hyperparameters = list(product(*[lr_s, batch_sizes]))
print(hyperparameters)
training_statistics = []
for lr, batch_size in hyperparameters:
    # config = AlbertConfig.from_json_file(CONFIG_FILE)
    # model = AlbertModel(config)
    # model = load_tf_weights_in_albert(model, config,ALBERT_PRETRAIN_CHECKPOINT)
    PRE_TRAINED_MODEL_NAME_OR_PATH = '/content/gdrive/My Drive/ALBERTimplementation/AGnewsmodel'
    model = AlbertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME_OR_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False)
    
    print('START----',model,'END---')
   
    # Tell pytorch to run this model on the GPU.
    model.cuda()

    # The DataLoader needs to know our batch size for training, so we specify it 
    # here. For fine-tuning ALBERT on a specific task, the authors recommend a batch 
    # size of 16 or 32.

    # Create the DataLoaders for our training and validation sets.
    # We'll take training samples in random order. 
    train_dataloader = DataLoader(
                train_dataset,  # The training samples.
                sampler = RandomSampler(train_dataset), # Select batches randomly
                batch_size = batch_size # Trains with this batch size.
            )

    # For validation the order doesn't matter, so we'll just read them sequentially.
    validation_dataloader = DataLoader(
                val_dataset, # The validation samples.
                sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                batch_size = batch_size # Evaluate with this batch size.
            )
    
    # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
    # I believe the 'W' stands for 'Weight Decay fix"
    optimizer = AdamW(model.parameters(),
                      lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                      eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                    )
    

    # Number of training epochs. The BERT authors recommend between 2 and 4. 
    # We chose to run for 4, but we'll see later that this may be over-fitting the
    # training data.
    epochs = 3

    # Total number of training steps is [number of batches] x [number of epochs]. 
    # (Note that this is not the same as the number of training samples).
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)

    seed_val = 42
    set_random(seed_val)
    
    print("Training with hyperparameters: batch size={}, lr={}".format(batch_size, lr))
    training_stats = train_model(train_dataloader, optimizer, epochs)
    training_statistics.append(training_stats)
    print_training_stats(training_stats)

[(2e-05, 32)]


Some weights of the model checkpoint at /content/gdrive/My Drive/ALBERTimplementation/AGnewsmodel were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from t

START---- AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(20001, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_fea

In [ ]:
model_save_name = 'finetuned_Albert.bin'
path = F"/content/gdrive/My Drive/ALBERTimplementation/model-fine-train/"+model_save_name
torch.save(model.state_dict(), path)